# Assignment Applied Machine Learning BSc IK 

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: **alleen de hoofdletter** (iets als  `A` of `B` dus)

__Namen__: Tycho Stam,
    
__Emails__: tycho.stam@student.uva.nl,

__Student ids__ : 13303147,

__Groep__ : B

## Toelichting

* Een aantal opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Applied Machine Learning W3

In the assignment for this week the focus will be on evaluation, where we take a look at confusion matrices, different metrics, and evaluation techniques such as cross validation and confidence intervals. As always, the exercises are mostly independent of each other, so if you get stuck somewhere, try doing a different part of the assignment, and come back to it later.

## Index

1. [Confusion Matrices](#conf_mat)
2. [Evaluation DIY](#eval1)
3. [Evluation with sklearn](#eval2)
4. [Cross Validation & Confidence Intervals](#cross_val)
5. [Baselines](#baselines)
6. [More Theory Questions](#theory)
7. [contagiousness Vs. Deadliness](#contag)

In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns

# testing
from nose.tools import assert_count_equal, assert_equal, assert_almost_equals
from numpy.testing import *
from numpy.testing import assert_equal
from pandas.testing import assert_frame_equal

# Please do not remove this: 
np.random.seed(31415)

<a id="conf_mat" />

# Confusion Matrix

In the class we looked at the following example Confusion Matrix with multiple classes:


* example  with 3 classes, N=1000
    * priors 700,200,100
    * diagonal : 600,50,10
    * predicted classes: 900,80,20
* Compute, precision, recall, F1 per class and  accuracy, macro values, weighted values over all classes.

## Your turn

* Create an example yourself, with maybe more classes and completely different numbers.
* Compute precision and recall per class alongside the table, indicate very very clearly what you are doing, and the order of things.
* Do it on paper with pencil and nice colors, make a photo, put it on the web, and create a hyperlink in the answer cell to your picture(s).

YOUR ANSWER HERE

<a id="eval1" />

# Evaluating a model

In this exercise we will be creating a complete ML pipeline where we load in a dataset, prepare it for classification by splitting it in train and test, run a classifier, and evaluate the results of this classifier.

We start by loading in the dataset, this code is already given for you.


In [2]:
def loadfile():
    if 'faults.csv.gz' in os.listdir():
        return 'faults.csv.gz'
    elif os.path.exists('../../data/Week1/'):
        return '../../data/Week3/faults.csv.gz'
    elif os.path.exists('../../../data/Week1/'):
        return '../../../data/Week3/faults.csv.gz'
    
df= pd.read_csv(loadfile())
print(df.shape)
df.head()

(1941, 34)


,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


The above dataframe contains data about the faults of machines. It is a multiclass classification problem with 6 fault classes and a 7th "other" class. It has 27 explanatory variables. Let's make a piechart of the data so that we can get a bit of a feel for the data.

In [3]:
classes= ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults']
df[classes].sum().plot(kind='pie', title='Distribution of the classes among the 1941 instances.');
df[classes].sum().sort_values(ascending=False)

Other_Faults    673
Bumps           402
K_Scatch        391
Z_Scratch       190
Pastry          158
Stains           72
Dirtiness        55
dtype: int64

Next up is actually running the experiment, which we are going to to in the steps provided below.

### Start up
1. Add a column `target` to `df` which contains the label names from the `classes` as values and a row has label `X` as value in that column iff `df[X]==1`. (maybe there is a `numpy` function you can use?)
2. Drop the original columns of the labels from the dataframe. 
3. Do a stratified 70-30 data train test split. 
    * Indicate and check visually that the data is split stratified, so each class also has a 70-30 split.
4. Create `y_pred` using KNN. You may want to normalize the data first.

### Evaluate

1. Create the confusion table based on `y_pred` and `y_test` from scratch, that is not using any `sklearn` functions. 
    2. Also show it nicely using `sns.heatmap`. 
2. Compute precision, recall and F1 for each label.
3. Compute accuracy.
4. Compute Macro, micro and weighted versions of precision, recall and F1.
5. Create at least two non-learned rule based baseline predictions, compute evaluation metrics and put these together with the scores for your KNN model into a clear table.

### Starting up

In [4]:
# start by adding the new column
df['target'] = None
# Fill in the right code in the line below
df['target'] = np.argmax(df[classes].values, axis=1)
new_df = df.drop(classes, axis=1)
# after adding the column, make matrices of the data for easy use
faults_X, faults_y = np.array(new_df.iloc[:, :-1]), np.array(new_df['target'])

T=pd.Series(new_df.target).value_counts()
T.plot(kind='pie', title='Distribution of the classes among the 1941 instances.');
T

6    673
5    402
2    391
1    190
0    158
3     72
4     55
Name: target, dtype: int64

In [5]:
assert_equal(new_df['target'].min(), 0)
assert_equal(new_df['target'].max(), 6)

Now that we have added the target column to the data we can create the train test split of the data, you may use `sklearn` for this.

In [6]:
# start up split
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(faults_X, faults_y, test_size=0.30, stratify=faults_y)

In [7]:
# run the following print statements, if the proportions are similar
# it means that stratification is working correctly

print("Distribution of labels in the train set")
print(pd.Series(train_y).value_counts(normalize=True))
print()
print("Distribution of labels in the test set")
print(pd.Series(test_y).value_counts(normalize=True))


Distribution of labels in the train set
6    0.346834
5    0.206922
2    0.201767
1    0.097938
0    0.081738
3    0.036819
4    0.027982
dtype: float64

Distribution of labels in the test set
6    0.346484
5    0.207547
2    0.200686
1    0.097770
0    0.080617
3    0.037736
4    0.029160
dtype: float64


Next up is running the K Nearest Neighbours algorithm on the data, complete the function below, where you are allowed to use the sklearn function `KNeighborsClassifier`, we do this in a fucntion so that we can experiment with parameters later if we want to.

In [8]:
# start up knn and compute the predictions
from sklearn.neighbors import KNeighborsClassifier

def knn_predict(train_features, train_labels, test_features, num_neighbors=5):
    y_pred = None
    classifier = KNeighborsClassifier(n_neighbors=num_neighbors)
    classifier.fit(train_features, train_labels)
    y_pred = classifier.predict(test_features)

    
    return np.array(y_pred)

# DONT CHANGE THE LINE BELOW
y_pred = knn_predict(train_X, train_y, test_X, num_neighbors=5)

### Evaluation

Next up is creating the confusion matrix of the gold standard labels and our predictions. You are not allowed to use sklearn functions for this. Also, make sure that your code will work for any input of a gold standard array and a predicted array, regardless of the number of classes, so do not hardcode this!

**Hints**
We are following the definition of the confusion matrix where a row is the predicted class and the column is an actual class, like the first example in the HC slides with apples and oranges. (and the opposite of the wikipedia definition)

In [9]:
# we expect the rows in the matrix to be predicted classes, and the columns to be the numbers for the gold
# standard classses, this is different from the sklearn representation!

def construct_confusion_matrix(gold_standard_labels, predicted_labels):    
    # we will give you the first part, we create an empty square matrix
    # the size of the number of labels in the gold standard, understand why?
    number_of_classes = len(np.unique(gold_standard_labels))
    confusion_matrix = np.zeros((number_of_classes, number_of_classes))
    
    for x in range(len(predicted_labels)):
        confusion_matrix[predicted_labels[x], gold_standard_labels[x]] = (confusion_matrix[predicted_labels[x],
                                                                                           gold_standard_labels[x]] + 1)

    #WRITE YOUR CODE HERE
    return confusion_matrix

matrix = construct_confusion_matrix(test_y, y_pred)

In the cell below make a nice confusion matrix here using `y_pred` and `sns.heatmap`

In [10]:
sns.heatmap(matrix, annot=True)

<AxesSubplot:title={'center':'Distribution of the classes among the 1941 instances.'}>

### Calculating Accuracy, Precision Recall and F1 through the confusion matrix

Next up is calculating accuracy, precision, recall and F1. For this exercise you are not allowed to use the functions from sklearn, but you have to implement them yourself using the confusion matrix. Your task is to fill in the functions below, where the behaviour is similar to sklearn, where you can give in an 'average' parameter, which calculates the metrics with different weighting schemes.

**Hints**
- Try to think about precision and recall in terms of rows and columns in the confusion matrix, this should make the calculations relatively straightforward to implement in numpy.
- you can use `np.diag` to get the values on the diagonal of the confusion matrix.
- For F1 you can use the functions of precision and recall you wrote, but use the `None` option for weighting and calculate the different weights using these arrays, to avoid successive rounding errors in your results. Then use `np.nan_to_num` on the raw F1 array in case we have precision and recall both equal to 0.

In [11]:
# complete the below functions, where you implement the following types for each metric:
# None: return the metric for each label separately, thus returning an array of size num_classes
# 'macro': return the macro weighted metric
# 'micro': returns the micro weighted metric
# 'weighted': returns the class weighted metric

# As a warmup, calculate accuracy over the entire test set
def accuracy(confusion_matrix): 
    accuracy = sum(np.diag(confusion_matrix)) / confusion_matrix.sum()
    return accuracy

def precision(confusion_matrix, average): 
    assert average in [None, 'macro', 'micro', 'weighted']
    precision = np.diagonal(confusion_matrix) / np.sum(confusion_matrix, axis= 1)
    
    if average ==  None:           
        return precision
    
    if average == 'micro':
        return accuracy(confusion_matrix)
    
    if average == 'macro':
        return np.mean(precision)
    
    if average == 'weighted':   
        weighted = np.sum(confusion_matrix, axis=1)
        mean_score = np.diagonal(confusion_matrix) / np.sum(confusion_matrix,axis = 0)
        return np.average(mean_score, weights=weighted)


def recall(confusion_matrix, average): 
    assert average in [None, 'macro', 'micro', 'weighted']
    recall = np.diagonal(confusion_matrix) / np.sum(confusion_matrix, axis= 0)
    
    if average == None:
        return recall
    
    if average == 'micro':
        return accuracy(confusion_matrix)
    
    if average == 'macro':
        return np.mean(recall)
    
    if average == 'weighted':
        weighted = np.sum(confusion_matrix, axis=0)
        mean_score = np.diagonal(confusion_matrix) / np.sum(confusion_matrix,axis = 0)
        return np.average(mean_score, weights=weighted)
    
def f1(confusion_matrix, average):
    assert average in [None, 'macro', 'micro', 'weighted']
    
    numerator = precision(confusion_matrix, None) * recall(confusion_matrix, None)
    denominator = precision(confusion_matrix, None) + recall(confusion_matrix, None)
    
    f1 = np.nan_to_num(2 * (numerator / denominator))
    
    if average == None:     
        return np.nan_to_num(f1)
    
    if average == 'micro':
        return accuracy(confusion_matrix)
    
    if average == 'macro':
        return np.mean(f1)
    
    if average == 'weighted':
        return np.sum(f1 * np.sum(confusion_matrix, axis=1)) / np.sum(np.sum(confusion_matrix, axis=1))



In [12]:
test_matrix = np.array([[1, 2, 0],
                       [0, 3, 1],
                       [0, 5, 2]])

assert_equal(type(precision(test_matrix, None)), np.ndarray)
assert_equal(type(recall(test_matrix, None)), np.ndarray)
assert_equal(type(f1(test_matrix, None)), np.ndarray)


In [13]:
# Compute Macro, micro and weighted versions of precision, recall and F1.
# TODO: also use round to check here to make sure that we dont get small rounding differences
test_matrix = np.array([[1, 2, 0],
                       [0, 3, 1],
                       [0, 5, 2]])

assert_equal(type(precision(test_matrix, "macro")), np.float64)
assert_equal(type(recall(test_matrix, "macro")), np.float64)
assert_equal(type(f1(test_matrix, "macro")), np.float64)

assert_equal(type(precision(test_matrix, "micro")), np.float64)
assert_equal(type(recall(test_matrix, "micro")), np.float64)
assert_equal(type(f1(test_matrix, "micro")), np.float64)

assert_equal(type(precision(test_matrix, "weighted")), np.float64)
assert_equal(type(recall(test_matrix, "weighted")), np.float64)
assert_equal(type(f1(test_matrix, "weighted")), np.float64)


<a id="eval2" />

# Evaluate again

1.  Figure out how you can make a nice evaluation report using `sklearn`. Make such a report with `sklearn`.
2. Then make the same report using yoiur own functions. You need to add the support, and possibly more. Do that of course.

#tip: nice sklearn summary table that gives precision, recall, f1, support. Use that ;)
Also make the same report with your own functions, think about support as well

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
average = [None, 'macro', 'micro', 'weighted']

for item in average:
    print("Precision {}: {}".format(item, precision(matrix, item) == precision_score(test_y, y_pred, average=item)))
    print("Recall {}: {}".format(item, recall(matrix, item) == recall_score(test_y, y_pred, average=item)))
    print("F1 {}: {}\n".format(item, f1(matrix, item) == f1_score(test_y, y_pred, average=item)))

Precision None: [ True  True  True  True  True  True  True]
Recall None: [ True  True  True  True  True  True  True]
F1 None: [ True  True  True  True  True  True  True]

Precision macro: True
Recall macro: True
F1 macro: True

Precision micro: True
Recall micro: True
F1 micro: True

Precision weighted: False
Recall weighted: True
F1 weighted: False



<ipython-input-11-8cbb0e59f368>:55: RuntimeWarning: invalid value encountered in true_divide
  f1 = np.nan_to_num(2 * (numerator / denominator))
<ipython-input-11-8cbb0e59f368>:55: RuntimeWarning: invalid value encountered in true_divide
  f1 = np.nan_to_num(2 * (numerator / denominator))
<ipython-input-11-8cbb0e59f368>:55: RuntimeWarning: invalid value encountered in true_divide
  f1 = np.nan_to_num(2 * (numerator / denominator))
<ipython-input-11-8cbb0e59f368>:55: RuntimeWarning: invalid value encountered in true_divide
  f1 = np.nan_to_num(2 * (numerator / denominator))


In [15]:
from sklearn.metrics import classification_report

print(classification_report(test_y, y_pred, target_names=classes))

              precision    recall  f1-score   support

      Pastry       0.09      0.06      0.08        47
   Z_Scratch       0.20      0.16      0.18        57
    K_Scatch       0.84      0.79      0.81       117
      Stains       0.00      0.00      0.00        22
   Dirtiness       0.48      0.59      0.53        17
       Bumps       0.36      0.42      0.39       121
Other_Faults       0.47      0.50      0.48       202

    accuracy                           0.46       583
   macro avg       0.35      0.36      0.35       583
weighted avg       0.45      0.46      0.45       583



<a id="cross_val" />

# Cross validation

* Do 5 fold cV for the KNN Classifier that we used in the previous exercise and the faults dataset, using the original `faults_X` and `faults_y arrays` we defined in the beginning.
* Does that make sense for the small classes? Think, experiment, look, and make a decision.
* Then run it. 
* Show for each class the distribution of the F1 values. Choose yourself how.
* Compute the confidence intervals. 
* What can you conclude?

For this exercise your are allowed to use functions from sklearn, and also use the `f1_score` from sklearn.

**Hints**
- If you calculate the F1 for each class individually, you can use a numpy array to store the results for each split, and then use numpy functions to easily calculate the confidence intervals.

Remember, we calculate confidence intervals for a class using $\mu \pm \frac{\sigma}{\sqrt{k}}$ where k is the number of folds.

In [16]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt

# Don't forget about the `knn_predict` function you wrote!
# Used code from https://www.datacamp.com/tutorial/k-nearest-neighbor-classification-scikit-learn

CV = 5
k_values = [i for i in range (1,20)]
scores = []


for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, faults_X, faults_y, cv=CV)
    scores.append(np.mean(score))
    
sns.lineplot(x = k_values, y = scores)
plt.xlabel("K Values")
plt.ylabel("Accuracy Score")

Text(77.92222222222227, 0.5, 'Accuracy Score')

A KNN of 15 gives the highest accuarcy.

<a id="baselines" />

# Baselines

Next we are going to implement some very simple baselines that do not use any information from the features, but instead rely only on the label. As with the previous functions, make sure that they work for any label column, so don't hardcode things! As these functions don't rely on the input features , the only input you get is the y label column, and a parameter that specifies the number of samples in the test set.

## Majority class

* Always choose the majority class

## Random 

* With $C$ classes, let each class have $\frac{1}{C}$ chance.

## Weighted Random 

*  Let each class have $P(C)$ chance.
    * $P(C)$ = fraction of C-instances in the population.
    
## Compute 

* Make a clear table with macro, micro, and weighted versions precision, recall and F1 for these three baselines for the `faults` dataset.  Clearly indicate how and why you decided to calculate things the way you did.

In [17]:
# Majority class
from scipy.stats import mode # what does this function return, how can you use it?

def majority_class(input_labels, test_size):
    '''Always choose the majority class'''
    
    output_labels = mode(input_labels)[0]
    output_labels = np.tile(output_labels, test_size)
    
    return np.array(output_labels)

In [18]:
assert_equal(len(majority_class([0, 1, 0, 1], test_size=20)), 20)

In [19]:
from numpy.random import randint

def random(input_labels, test_size): 
    '''With  𝐶 classes, let each class have  1/𝐶 chance.'''
    
    labels = np.unique(input_labels)    
    output_labels = np.random.randint(labels[-1] + 1, size=test_size)
    
    return np.array(output_labels)

In [20]:
assert_equal(len(random([0, 1, 0, 1], test_size=20)), 20)

In [21]:
# Hint: have a look at the numpy.random.choice function with a distribution
# Another hint: you can get counts of unique values with np.unique (look at the documentation)
# which you can use to the probabilities for your distribution
from numpy.random import choice

def weighted_random(input_labels, test_size):
    '''Let each class have  𝑃(𝐶) chance. 𝑃(𝐶) = fraction of C-instances in the population'''
    # count all the labels
    labels, counts = np.unique(input_labels, return_counts=True)
    label_count = dict(zip(labels, counts))
    
    # calculate weight
    weighted = [(label_count[i] / len(input_labels)) for i in label_count]
    
    output_labels = np.random.choice(labels[-1] + 1, size=test_size, p=weighted)
    
    return np.array(output_labels)

In [22]:
assert_equal(len(random([0, 1, 0, 1], test_size=20)), 20)

Now construct your table with results using the functions we just wrote above

In [43]:
precision_score(majority_class(faults_y, len(y_pred)), y_pred, average=None)

for item in average:
    print("Precision Majority Class {}: {}".format(item, precision_score(majority_class(faults_y, len(y_pred)),
                                                                         y_pred, average=item)))
    print("Precision Random Class {}: {}".format(item, precision_score(random(faults_y, len(y_pred)),
                                                                         y_pred, average=item)))
    print("Precision Weighted Random Class {}: {}".format(item, precision_score(weighted_random(faults_y, len(y_pred)),
                                                                         y_pred, average=item)))
    
#     print("Recall {}: {}".format(item, recall(matrix, item) == recall_score(test_y, y_pred, average=item)))
#     print("F1 {}: {}\n".format(item, f1(matrix, item) == f1_score(test_y, y_pred, average=item)))

Precision Majority Class None: [0. 0. 0. 0. 0. 0. 1.]
Precision Random Class None: [0.15625    0.22222222 0.19090909 0.05882353 0.0952381  0.13286713
 0.11162791]
Precision Weighted Random Class None: [0.0625     0.06666667 0.21818182 0.05882353 0.04761905 0.20979021
 0.36744186]
Precision Majority Class macro: 0.14285714285714285
Precision Random Class macro: 0.16097204656785463
Precision Weighted Random Class macro: 0.17485597412922993
Precision Majority Class micro: 0.3687821612349914
Precision Random Class micro: 0.137221269296741
Precision Weighted Random Class micro: 0.22298456260720412
Precision Majority Class weighted: 1.0
Precision Random Class weighted: 0.12630568458924843
Precision Weighted Random Class weighted: 0.2390374745190454


In [42]:
precision_score(majority_class(faults_y, len(y_pred)), y_pred, average=None)
precision_score(random(faults_y, len(y_pred)), y_pred, average=None)
precision_score(weighted_random(faults_y, len(y_pred)), average=None)

precision_score(majority_class(faults_y, len(y_pred)), y_pred, average='macro')
precision_score(random(faults_y, len(y_pred)), y_pred, average=None)
precision_score(weighted_random(faults_y, len(y_pred)), average=None)

ValueError: diag requires an array of at least two dimensions

In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate
SIZE = 20

row_head=['majority class', 'random', 'weighted random']
merged_array = np.array([majority_class(test_y, test_size=SIZE), random(test_y, test_size=SIZE),
                         weighted_random(test_y, test_size=SIZE)]).T

table = tabulate(merged_array , row_head, tablefmt="fancy_grid")
print(table)

<a id ="theory" />

# Part 3: theory

The following questions are about precision, recall and accuracy.

Suppose you have a disease that occurs in 1 in a 100 people, and you have a test with an accuracy of .95. Meaning that in 95 percent of the case you correctly predict whether someone has the disease or not: The sum of the elements on the diagonal divided by the sum of all elements in the confusion matrix is .95.

1. What is de maximum (variable `maxR`) and minimal (`minR`) *recall* for the disease-class with these numbers?
2. What is de maximum (variable `maxP`) and minimal (`minP`) *precision* for the disease-class with these numbers?

**Hint** Construct a confusion matrix where you try different settings, make sure the numbers add up to the right totals.

## OLD DUTCH TEXT

Deze vragen gaan over precisie, recall en accuraatheid.

Stel je hebt een ziekte die bij 1 op de 100 mensen voorkomt. En je hebt een test met een accuraatheid van .95. Dit laatste betekent dat je in 95% van de gevallen juist voorspelt of iemand de ziekte heeft of niet. Anders gezegd: de som van de elementen op de diagonaal van de confusion matrix gedeeld door het totaal aantal elementen is .95.

1. Wat is de maximale(variabele `maxR`) en minimale (`minR`) *recall* voor de ziekte-klasse bij deze gegevens?
2. Wat is de maximale (`maxP`) en minimale (`minP`) *precision* voor de ziekte-klasse bij deze gegevens?

**Hint** Maak een confusion matrix waarin je verschillende settings uitprobeert. Zorg er steeds voor dat alles netjes optelt. 



In [ ]:
def create_confusion_df(total, occurance, accuracy=0.95, round_number=0):
    '''Create a confusion df'''

    data = np.empty((3, 3))
    df = pd.DataFrame(data, columns=[['Heeft ziekte', 'Heeft geen ziekte', '_']],
                      index=[['Positief getest op de ziekte', 'Negatief getest op de ziekte', '_' ]])

    df.iloc[2,2] = total
    df.iloc[2,0] = total * occurance
    df.iloc[2,1] = total * (1 - occurance)

    df.iloc[0,0] = total * occurance * accuracy
    df.iloc[0,1] = total * occurance * (1 - accuracy)
    df.iloc[1,0] = total * (1 - occurance) * (1 - accuracy)
    df.iloc[1,1] = total * (1 - occurance) * accuracy

    df.loc['Positief getest op de ziekte', '_'] = (df.iloc[0,0] + df.iloc[0,1])
    df.loc['Negatief getest op de ziekte', '_'] = (df.iloc[1,0] + df.iloc[1,1])

    return df.round(round_number)

In [ ]:
def recall_df(df):
    return df.iloc[0,0] / (df.iloc[0,0] + df.iloc[1,0])

def precision_df(df):
    return df.iloc[0,0] / (df.iloc[0,0] + df.iloc[0,1])

In [ ]:
create_confusion_df(total=50000, occurance=0.01, accuracy=0.95)

In [ ]:
create_confusion_df(total=100000, occurance=0.01, accuracy=0.95)

In [ ]:
minP = 0.05  # precisie
maxP = 0.95
minR = 0.01  # recall
maxR = 0.99

In [ ]:
for m in {minP, maxP, minR, maxR}:
    assert 0 <= m <= 1

## Variant 

Dezelde vraag als boven, maar nu hebben 5 op de 100 mensen de ziekte en hebben we juist een accuraatheid van .99

In [ ]:
create_confusion_df(total=10000, occurance=0.05, accuracy=0.99)

In [ ]:
minP = 0.01  # precisie
maxP = 0.99
minR = 0.05  # recall
maxR = 0.95

#WRITE YOUR CODE HERE

In [ ]:
for m in {minP, maxP, minR, maxR}:
    assert 0 <= m <= 1

<a id="contag" />

# Part 4: What is worse: contagiousness of or deadliness?


Source: <https://www.nrc.nl/nieuws/2020/12/31/verdachte-virusvariant-met-opvallend-veel-mutaties-a4025804>

> What is worse: contagiousness or deadliness? At first glance you might think the second is. In the media there is thus also a sense of relief that the new variant of the virus is more more contagious, but less deadly than the existing variants. But is this justified? Adam Kucharski of the London School of Hygiene and Tropical Medicine [argued on Twitter](https://twitter.com/AdamJKucharski/status/1343567425107881986) that it is not.He gave the following numerical example

>Suppose R = 1.1. This means that on average ten people will infect eleven other people. Now suppose the generation time is 6 days i.e. it takes 6 days for those ten people to have infected these eleven people. This in 30 days this 'tree of contagion' splits 5 times, once each six days. Now suppose that of all people that become infected 0.8 percent die of the illness and the at the start we started with on day zero with 10.000 infected people. Then after one month we have $10.000 × (1,1)^5 × 0,8\% = 129$ people that have died.

>What happens when a mutation causes the virus to become 50 percent more deadly? If we will in the above formula with the changed deadliness we get the following: (1,1)^5 × (0,8\% × 1,5) = 193 $ people dying from the virus.

>Now what happens when the virus becomes 50 percent more contagious? Again, we can change the contagiousness in the formula and recalculate: $10.000 × (1,1 × 1,5)^5 × 0,8\% = 978$, meaning we now have almost a thousand people dying from the virus

>How is this possible? The deadliness of the virus is linearly correlated with the number of infrected people, while the number of infected people is related exponentially with the number of infected people, which we can also observe in the formula.

* Run the code below and try to understand what happens, do you understand why there is a difference in the number of dead in period 0, why it is the same for period 1, and why it starts to diverge after period 1?
* Check that the number of dead at the end of the month in the article is the same as the number at week 5 in the table.
* And see how we can very easily make a plot from spreadsheet data like this? We will be using this more in the coming weeks and explore even more cool ways of plotting data and results.

## Is it actually correct?
1. The formula is of course correct, however the outcome is not. Because if you have an R of 1.1 for a month you ahve had 6 'generations', and the total number of infected people is much higher than $10.000 x (1,1)^5$. Calculate how much more in the fuction `aantal_geinfecteerden_na_periodes`.

2. The same goes for the number of people that have died after a month, calculate this as well.



# OLD DUTCH VERSION
>Wat is erger: besmettelijkheid of dodelijkheid? Op het eerste gezicht denk je het tweede. In de media is dan ook opluchting te bespeuren dat het virus weliswaar besmettelijker is, maar geen ernstiger ziekteverloop veroorzaakt. Maar is dat terecht? Adam Kucharski van de London School of Hygiene and Tropical Medicine [betoogde op Twitter](https://twitter.com/AdamJKucharski/status/1343567425107881986) van niet. Hij gaf het volgende getallenvoorbeeld:

>Stel, R is 1,1. Dat wil zeggen dat tien besmette personen gemiddeld elf anderen besmetten. Stel, de generatietijd is zes dagen: het duurt zes dagen tot die tien mensen elf anderen hebben besmet, dus in 30 dagen vertakt die ‘besmettingsboom’ zich vijf keer. Stel, van alle besmette mensen overlijdt 0,8 procent. En stel, op tijdstip nul zijn 10.000 mensen besmet. Dan levert dat na een maand $10.000 × (1,1)^5 × 0,8\% = 129$ doden op.

>Wat gebeurt er dan als het virus door een mutatie 50 procent dodelijker wordt? Dan zijn na een maand $10.000 x (1,1)^5 × (0,8\% × 1,5) = 193 $
mensen overleden.

>Als het virus door een mutatie echter 50 procent besmettelijker wordt, dan ligt het dodental bijna vijf keer zo hoog: $10.000 × (1,1 × 1,5)^5 × 0,8\% = 978$ doden.


>Hoe dat kan? De dodelijkheid correleert lineair met het aantal doden, maar voor besmettelijkheid is dat verband exponentieel.

* Run onderstaande code. Snap wat er gebeurt. Snap je het verschil in aantal doden voor periode 0. En waarom is het gelijk in periode 1, en gaat het daarna (steeds sneller) uit elkaar lopen?
* Check dat het aantal doden aan het eind van de maand in het artikel hetzelfde is als in de tabel bij periode 5.
* En kijk eens hoe makkelijk je een plotje van zo'n spreadsheet maakt. Ideaal toch? Later in de cursus leren we hier meer over.

## Klopt het wel? (1pt)

1. De formule klopt natuurlijk, maar de uitkomst niet. Want als je een maand met een R van 1.1 zit heb je 6 "generaties", en dus in totaal veel meer besmette mensen gehad dan $10.000 x (1,1)^5$.  **Hoeveel meer dan?** **Bereken dat in de functie `aantal_geinfecteerden_na_periodes`.**
2. En dus hebben we na een maand ook een ander aantal overledenen.  Bereken dat eens.


In [55]:
# zoals in het artikel
def aantaldoden(aantalbesmet=10**4,
                overlijdingsratio=.008,
                R=1.1,
                hoeveelerger=1.5,
                aantalperiodes=5):
    return {
        'dodelijker':
        round(aantalbesmet * (R)**aantalperiodes * overlijdingsratio *
              hoeveelerger),
        'besmettelijker':
        round(aantalbesmet * (R * hoeveelerger)**aantalperiodes *
              overlijdingsratio)
    }


# 50 % erger
A = {P: aantaldoden(hoeveelerger=1.5, aantalperiodes=P) for P in range(0, 6)}
A = pd.DataFrame(A).T
A.index.name = 'Periode'
A.plot(kind='bar', title='Aantal doden van geinfecteerden in periode x.')
A

,dodelijker,besmettelijker
Periode,,
0,120,80
1,132,132
2,145,218
3,160,359
4,176,593
5,193,978


In [53]:
def aantal_geinfecteerden_na_periodes(
    aantalperiodes=6,
    aantalbesmet=10**4,
    R=1.1,
):
    '''Geef het totaal aantal geinfecteerde mensen terug na aantalperiodes generaties/periodes.
    Rond af op een geheel getal.'''

    return round(aantalbesmet * (R)**aantalperiodes)
    


B = pd.Series({
    P: round(aantal_geinfecteerden_na_periodes(aantalperiodes=P))
    for P in range(21)
})
B.plot(kind='bar')
B

0     10000
1     11000
2     12100
3     13310
4     14641
5     16105
6     17716
7     19487
8     21436
9     23579
10    25937
11    28531
12    31384
13    34523
14    37975
15    41772
16    45950
17    50545
18    55599
19    61159
20    67275
dtype: int64

In [54]:
assert isinstance(aantal_geinfecteerden_na_periodes(), int)



After 6 generations there are 17716 people infected. Counting with the deased. 

## Altered dataframe (1pt)

Example the code above again, where we construct teh dataframe `A`. Alter the code such that you print the correct numbers acorrding to the text above, and store this in dataframe `AA`. `AA` will have the same shape and index as `A`, just different numbers

* Also observe the difference between the titles of the two plots.


## Aangepast dataframe (1pt)

Bekijk de code hierboven nog eens waarin het Dataframe `A` gemaakt werd. Pas die code aan, zodat je daarmee het dataframe `AA` maakt dat de juiste aantallen print. `AA` heeft dezelfde *shape* als `A`, en ook dezelfde index en kolom namen. Alleen de waardes zijn anders. 

* Apprecieer ook het verschil in de titels tussen de twee plotjes.

In [ ]:
AA = A  # Change with your answer

#WRITE YOUR CODE HERE

T = 'Totaal aantal doden na x periodes van 6 dagen.'
AA.plot(kind='bar', title=T)
print(AA)


#WRITE YOUR CODE HERE

In [ ]:
assert_equal(AA.shape, (6, 2))  # 2 kolommen, 6 rijen
assert_equal(list(AA.index) == list(range(6)),
             True)  # de index van AA is [0,1,2,3,4,5]

